# Curve fitting with `lmfit`

In this section, we will cover basic curve fitting using [lmfit](https://lmfit.github.io/lmfit-py/) for reference purposes. For detailed information, please refer to the [lmfit documentation](https://lmfit.github.io/lmfit-py/).

If you are already familiar with [lmfit](https://lmfit.github.io/lmfit-py/), you can skip to the [next section](./modelfit).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [ ]:
%config InlineBackend.figure_formats = ["svg", "pdf"]
plt.rcParams["figure.dpi"] = 96
plt.rcParams["image.cmap"] = "viridis"
plt.rcParams["figure.figsize"] = (4, 2.5)

Let's start by defining a model function and the data to fit.

In [ ]:
def poly1(x, a, b):
    return a * x + b


# Generate some toy data
x = np.linspace(0, 10, 20)
y = poly1(x, 1, 2)

# Add some noise with fixed seed for reproducibility
rng = np.random.default_rng(1)
yerr = np.full_like(x, 0.5)
y = rng.normal(y, yerr)

## Models

A lmfit model can be created by calling {class}`lmfit.Model <lmfit.model.Model>` with the model function and the independent variable(s) as arguments.

In [ ]:
import lmfit

model = lmfit.Model(poly1)
params = model.make_params(a=1.0, b=2.0)
result = model.fit(y, x=x, params=params, weights=1 / yerr)

result.plot()
result

By passing dictionaries to `make_params`, we can set the initial values of the parameters and also set the bounds for the parameters.

In [ ]:
model = lmfit.Model(poly1)
params = model.make_params(
    a={"value": 1.0, "min": 0.0},
    b={"value": 2.0, "vary": False},
)
result = model.fit(y, x=x, params=params, weights=1 / yerr)
_ = result.plot()

`result` is a {class}`lmfit.model.ModelResult` object that contains the results of the
fit. The best-fit parameters can be accessed through the `result.params` attribute.


:::{note}

Since all weights are the same in this case, it has little effect on the fit results. However, if we are confident that we have a good estimate of `yerr`, we can pass `scale_covar=True` to the `fit` method to obtain accurate uncertainties.

:::

In [ ]:
result.params

In [ ]:
result.params["a"].value, result.params["a"].stderr

The parameters can also be retrieved in a form that allows easy error propagation calculation, enabled by the [uncertainties](https://github.com/lmfit/uncertainties) package.

In [ ]:
a_uvar = result.uvars["a"]
print(a_uvar)
print(a_uvar**2)

## Composite models

Before fitting, let us generate a Gaussian peak on a linear background.

In [ ]:
# Generate toy data
x = np.linspace(0, 10, 50)
y = -0.1 * x + 2 + 3 * np.exp(-((x - 5) ** 2) / (2 * 1**2))

# Add some noise with fixed seed for reproducibility
rng = np.random.default_rng(5)
yerr = np.full_like(x, 0.3)
y = rng.normal(y, yerr)

# Plot the data
plt.errorbar(x, y, yerr, fmt="o")

A composite model can be created by adding multiple models together.

In [ ]:
from lmfit.models import GaussianModel, LinearModel

model = GaussianModel() + LinearModel()
params = model.make_params(slope=-0.1, center=5.0, sigma={"value": 0.1, "min": 0})
params

In [ ]:
result = model.fit(y, x=x, params=params, weights=1 / yerr)
result.plot()
result

How about multiple gaussian peaks? Since the parameter names overlap between the models, we must use the `prefix` argument to distinguish between them.

In [ ]:
model = GaussianModel(prefix="p0_") + GaussianModel(prefix="p1_") + LinearModel()
model.make_params()

For more information, see the [lmfit documentation](https://lmfit.github.io/lmfit-py/model.html).